# 로지스틱 회귀

### 데이터 준비하기

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


* 타깃 데이터에는 7개의 생선의 종류가 들어 있습니다. 타깃이 2개 이상의 클래스가 포함된 문제를 다중 분류(multi class claasification)이라고 부릅니다. 

In [3]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

print('fish_input: \n{}'.format(fish_input[:5]))
print('fish_target: \n{}'.format(fish_target[:5]))

fish_input: 
[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]
fish_target: 
['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


In [4]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [5]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

### k-최근접 이웃 분류기의 확률 예측
* 사이킷런에서는 편리하게도 문자열로 된 타깃값을 그대로 사용 가능 

In [6]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


* 분류 클래스

In [7]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [8]:
print(kn.predict(test_scaled[:5]))
print(test_target[:5])

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
['Perch' 'Smelt' 'Pike' 'Whitefish' 'Perch']


##### predict_proba()
* 클래스별 확률값을 확인
* 테스트 세트에 있는 처음 5개의 샘플에 대한 확률을 출력 

In [9]:
import numpy as np

prob = kn.predict_proba(test_scaled[:5])

for x in np.round(prob[:5], decimals=4):
    print('Bream: {}, Parkki: {}, Perch: {}, Pike: {}, Roach: {}, Smelt: {}, Whitefish: {}'
    .format(x[0], x[1], x[2], x[3], x[4], x[5], x[6]))

Bream: 0.0, Parkki: 0.0, Perch: 1.0, Pike: 0.0, Roach: 0.0, Smelt: 0.0, Whitefish: 0.0
Bream: 0.0, Parkki: 0.0, Perch: 0.0, Pike: 0.0, Roach: 0.0, Smelt: 1.0, Whitefish: 0.0
Bream: 0.0, Parkki: 0.0, Perch: 0.0, Pike: 1.0, Roach: 0.0, Smelt: 0.0, Whitefish: 0.0
Bream: 0.0, Parkki: 0.0, Perch: 0.6667, Pike: 0.0, Roach: 0.3333, Smelt: 0.0, Whitefish: 0.0
Bream: 0.0, Parkki: 0.0, Perch: 0.6667, Pike: 0.0, Roach: 0.3333, Smelt: 0.0, Whitefish: 0.0


* 테스트 세트의 4번째 값의 최근접 이웃의 클래스 출력
* 참고: 2차원 데이터를 넘기기 위해서 3:4 슬라이싱 사용 

In [10]:
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


---

### 로지스틱 회귀
* 로지스틱 회귀(logistic regression)는 이름은 회귀지만 분류 모델 
* 이 알고리즘은 선형 회귀와 동일하게 선형 방정식을 학습
* 참고 : https://velog.io/@arittung/DeepLearningStudyDay7

#### 1. 로지스틱 회귀로 이진 분류 수행하기

* 넘파이 배열은 True, False 값을 전달하여 행을 선택할 수 있습니다. 
* 이를 불리언 인덱싱(boolean indexing)이라고 합니다. 

In [11]:
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])

['A' 'C']


* 불리언 인덱싱을 사용하여, 도미(Bream)와 빙어(Smelt) 데이터 split

In [12]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
bream_smelt_indexes2 = (test_target == 'Bream') | (test_target == 'Smelt')

# bs: bream_smelt 약어
train_input_bs = train_scaled[bream_smelt_indexes]
train_target_bs = train_target[bream_smelt_indexes]

test_input_bs = test_scaled[bream_smelt_indexes2]
test_target_bs = test_target[bream_smelt_indexes2]

##### LogisticRegression 클래스

In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_input_bs, train_target_bs)

LogisticRegression()

In [14]:
print(lr.predict(test_input_bs[:5]))
print(test_target_bs[:5])

['Smelt' 'Bream' 'Smelt' 'Bream' 'Bream']
['Smelt' 'Bream' 'Smelt' 'Bream' 'Bream']


In [15]:
print(lr.predict_proba(test_input_bs[:5]))

[[3.95673649e-02 9.60432635e-01]
 [9.99418084e-01 5.81915885e-04]
 [2.57680368e-02 9.74231963e-01]
 [9.94091561e-01 5.90843851e-03]
 [9.93797733e-01 6.20226656e-03]]


* 어떤 클래스 값이 양성인지 확인 
* Bream -> 0
* Smelt -> 1

In [16]:
print(lr.classes_)

['Bream' 'Smelt']


In [17]:
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


* 로지스틱 회귀 모델이 학습한 방정식은 다음과 같다
* z = -0.404 * (Weight) - 0.576 * (Length) - 0.663 * (Diagonal) - 1.013 * (Height) - 0.732 * (Width) - 2.161

In [18]:
fish.head(0)

,Species,Weight,Length,Diagonal,Height,Width


* decision_function() 메서드로 z 값을 출력

In [19]:
decisions = lr.decision_function(test_input_bs[:5])
print(decisions)

[ 3.18937919 -7.44860256  3.63251459 -5.12544773 -5.0766189 ]


* scipy 안에 expit() 함수: 시그모이드 함수

In [20]:
from scipy.special import expit

sig = expit(decisions)
print(sig)

print(['Bream' if x < 0.5 else 'Smelt' for x in sig])


[9.60432635e-01 5.81915885e-04 9.74231963e-01 5.90843851e-03
 6.20226656e-03]
['Smelt', 'Bream', 'Smelt', 'Bream', 'Bream']


---

#### 2. 로지스틱 회귀로 다중 분류 수행하기
* 다중 분류도 LogisticRegression 클래스를 사용
* max_iter 매개변수에서 반복 횟수를 지정 (default = 100)
* 반복 횟수가 부족하면 경고가 발생

* LogisticRegression은 기본적으로 릿지 회귀와 같이 계수의 제곱을 규제 (L2 규제) 
* 릿지 회귀에서는 alpha 매개변수로 규제의 양을 조절 (alpha가 커지면 규제도 커진다) 
* C : LogisticRegression에서 규제를 제어하는 매개변수 (작을수록 규제가 커진다, alpha와 반대) 
* C의 기본값: 1
* 아래 문제에서, 규제를 조금 완화하기 위해 20으로 설정

In [21]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [22]:
print('예측 : {}'.format(lr.predict(test_scaled[:5])))
print('타깃 : {}'.format(test_target[:5]))

예측 : ['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']
타깃 : ['Perch' 'Smelt' 'Pike' 'Whitefish' 'Perch']


In [23]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [24]:
prob = lr.predict_proba(test_scaled[:5])
print(np.round(prob, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


* 다중 분류일 경우 선형 방정식은 어떤 모습일지 coef_와 intercept_의 크기를 출력

In [25]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


In [26]:
print('coef: ', lr.coef_, '\n')
print('intercept: ', lr.intercept_)

coef:  [[-1.49001999 -1.02912482  2.59345218  7.70357843 -1.2007022 ]
 [ 0.19618201 -2.01068627 -3.77976461  6.50491598 -1.99482485]
 [ 3.56279819  6.34356926 -8.48970993 -5.757574    3.79307177]
 [-0.10458121  3.603196    3.93067836 -3.61737079 -1.75069674]
 [-1.40061524 -6.07503319  5.25969396 -0.87219889  1.86043687]
 [-1.38526189  1.49214385  1.39226012 -5.67734222 -4.40097623]
 [ 0.62149812 -2.32406483 -0.9066101   1.71599149  3.69369137]] 

intercept:  [-0.09205181 -0.26290863  3.25101315 -0.14742819  2.65498377 -6.78783248
  1.38422419]


In [27]:
np.sum(test_scaled[0]*lr.coef_[1]) + lr.intercept_[1]

1.0322290972029498

* 다중 분류 클래스마다 z 값을 하나씩 계산
* 가장 높은 z 값을 출력하는 클래스가 예측 클래스
* 다중 분류는 소프트맥스(softmax) 함수를 사용하여 7개의 z값을 확률로 변환

* decision_function()
    * z1 ~ z7까지의 값 계산

In [28]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


* scipy는 소프트맥스 함수 제공 
* axis=1로 지정하여 각 행, 즉 각 샘플에 대해 소프트맥스를 계산 
* axis 매개변수를 지정하지 않으면, 배열 전체에 대해 소프트맥스를 계산

In [29]:
from scipy.special import softmax

prob = softmax(decision, axis=1)
print(np.round(prob, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


* 위 과정을 predict_proba() 메서드로 바로 구할 수 있다

In [30]:
print(np.round(lr.predict_proba(test_scaled[:5]), decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
